## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Hermanus,-34.42,19.23,60.55,70,0,5.75,clear sky,ZA,2020-08-02 20:19:04
1,1,Atuona,-9.80,-139.03,78.62,76,0,16.93,clear sky,PF,2020-08-02 20:19:04
2,2,Ushuaia,-54.80,-68.30,41.00,75,20,19.46,few clouds,AR,2020-08-02 20:19:04
3,3,Kantunilkin,21.09,-87.45,89.60,66,40,8.05,scattered clouds,MX,2020-08-02 20:19:05
4,4,Upernavik,72.79,-56.15,44.11,69,99,7.47,overcast clouds,GL,2020-08-02 20:19:05


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
1,1,Atuona,-9.80,-139.03,78.62,76,0,16.93,clear sky,PF,2020-08-02 20:19:04
3,3,Kantunilkin,21.09,-87.45,89.60,66,40,8.05,scattered clouds,MX,2020-08-02 20:19:05
8,8,Albany,42.60,-73.97,86.00,63,75,12.08,light rain,US,2020-08-02 20:19:05
9,9,Victoria,22.29,114.16,78.01,94,100,8.01,light rain,HK,2020-08-02 20:14:21
10,10,Praia Da Vitoria,38.73,-27.07,77.00,65,40,10.76,scattered clouds,PT,2020-08-02 20:19:05
17,17,Namatanai,-3.67,152.43,79.07,84,25,7.36,light rain,PG,2020-08-02 20:19:06
21,21,Seoul,37.57,126.98,78.80,94,90,11.77,thunderstorm,KR,2020-08-02 20:15:21
22,22,Butaritari,3.07,172.79,82.56,73,0,14.94,clear sky,KI,2020-08-02 20:19:06
24,24,Alice Town,25.72,-79.30,84.45,79,100,24.54,light rain,BS,2020-08-02 20:19:07
26,26,Marsh Harbour,26.54,-77.06,83.30,79,100,27.76,overcast clouds,BS,2020-08-02 20:19:07


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Country        1
Date           0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Country        0
Date           0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.62,clear sky,-9.80,-139.03,
3,Kantunilkin,MX,89.60,scattered clouds,21.09,-87.45,
8,Albany,US,86.00,light rain,42.60,-73.97,
9,Victoria,HK,78.01,light rain,22.29,114.16,
10,Praia Da Vitoria,PT,77.00,scattered clouds,38.73,-27.07,
17,Namatanai,PG,79.07,light rain,-3.67,152.43,
21,Seoul,KR,78.80,thunderstorm,37.57,126.98,
22,Butaritari,KI,82.56,clear sky,3.07,172.79,
24,Alice Town,BS,84.45,light rain,25.72,-79.30,
26,Marsh Harbour,BS,83.30,overcast clouds,26.54,-77.06,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace("", np.nan)
hotel_df = hotel_df.dropna()
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [10]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.62,clear sky,-9.80,-139.03,Villa Enata
3,Kantunilkin,MX,89.60,scattered clouds,21.09,-87.45,Posada Paraiso
9,Victoria,HK,78.01,light rain,22.29,114.16,Mini Hotel Central
10,Praia Da Vitoria,PT,77.00,scattered clouds,38.73,-27.07,Hotel Branco I
17,Namatanai,PG,79.07,light rain,-3.67,152.43,Namatanai Hotel
...,...,...,...,...,...,...,...
672,Arawa,PG,75.79,broken clouds,-6.21,155.54,Coastline Guest House
673,Siverek,TR,84.20,clear sky,37.76,39.32,Mercan Hotel
681,Barkhan,PK,85.93,clear sky,29.90,69.53,Sardar
682,Bubaque,GW,80.98,overcast clouds,11.28,-15.83,Casa Dora


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""
# <dt>Description</dt><dd>{Description}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))